In [47]:
import pandas as pd
import numpy as np
import joblib

In [3]:
train_base = pd.read_csv('data/train_base.csv')
test_a_base = pd.read_csv('data/test_a_base.csv')

train_op = pd.read_csv('data/train_op.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')

train_trans = pd.read_csv('data/train_trans.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

train_label = pd.read_csv('data/train_label.csv')

## base处理

In [37]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
train_base_no_nan = train_base.drop(columns=['service3_level'],axis=1)
train_base_no_nan.dropna(axis=0,inplace=True)
user = train_base_no_nan['user'].values
train_base_no_user = train_base_no_nan.drop(columns=['user'],axis=1)
category_value = train_base_no_user.select_dtypes('O').values
ohe_encoder = ohe.fit(category_value)
one_hot_value = ohe_encoder.transform(category_value).toarray()

In [48]:
joblib.dump(ohe_encoder,'one_encoder.pkl')

['one_encoder.pkl']

In [38]:
number_value = train_base_no_user.select_dtypes('int64').values
df_base = pd.DataFrame()
df_base['user'] = user
base = np.concatenate([one_hot_value,number_value],axis=1)
v_df = pd.DataFrame(base)
df_base = pd.concat([df_base,v_df],axis=1)

## OP处理

In [86]:
train_op.head()

,user,op_type,op_mode,op_device,ip,net_type,channel,ip_3,tm_diff
0,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:22.000000000
1,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:21.000000000
2,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:23.000000000
3,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:26.000000000
4,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:41.000000000


In [11]:
op_df = pd.DataFrame()
group = train_op.groupby(['user'])['user'].count()
op_df['user']=group.index
op_df['count']=group.values

In [12]:
count_column=train_op.columns[1:-1]
for column in count_column:
    op_df[column] = train_op.groupby(['user'])[column].apply(lambda x:len(set(x))).values

In [15]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

In [16]:
train_op['time_diff_sec'] = train_op['tm_diff'].apply(timedelta2sec)
op_df['time_std']=train_op.groupby('user')['time_diff_sec'].agg('std').values

In [155]:
op_df.head()

,user,count,op_type,op_mode,op_device,ip,net_type,channel,ip_3,time_std
0,Train_00000,102,11,12,3,4,4,4,4,125051.535485
1,Train_00001,18,3,3,1,2,3,3,2,385614.886428
2,Train_00002,8,3,3,1,1,3,3,1,139.053690
3,Train_00003,108,8,8,3,19,3,3,6,247893.797669
4,Train_00004,5,2,2,1,1,2,2,1,20.659138


## 交易处理

In [18]:
train_trans.head()

,user,platform,tunnel_in,tunnel_out,amount,type1,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,185784,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,391769,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,27740,674e8d5860bc033d,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,36689,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,30746,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 07:50:05.000000000


In [176]:
trans_df = pd.DataFrame()
group = train_trans.groupby(['user'])['user'].count()
trans_df['user']=group.index
trans_df['count']=group.values

In [177]:
count_column=list(train_trans.columns[1:4])+list(train_trans.columns[5:9])
for column in count_column:
    trans_df['trans_'+column] = train_trans.groupby(['user'])[column].apply(lambda x:len(set(x))).values

In [178]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=train_trans.groupby(['user'])['amount'].agg(operate).values

In [179]:
train_trans['time_diff_sec'] = train_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=train_trans.groupby('user')['time_diff_sec'].agg(operate).values

In [180]:
trans_df.fillna(0,inplace=True)

In [181]:
trans_df.head()

,user,count,trans_platform,trans_tunnel_in,trans_tunnel_out,trans_type1,trans_ip,trans_type2,trans_ip_3,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_00000,13,2,2,1,4,3,2,3,267542,24798,53330.307692,30746.0,65274.138488,2478067.0,1169773.0,1.553383e+06,1627246.0,3.970320e+05
1,Train_00001,2,2,2,1,2,2,2,2,36098,36098,36098.000000,36098.0,0.000000,1821027.0,289554.0,1.055290e+06,1055290.5,1.082915e+06
2,Train_00002,12,2,2,1,3,4,2,4,162423,24980,57329.583333,34541.5,49227.699521,2665430.0,575019.0,1.454428e+06,625491.0,1.069315e+06
3,Train_00003,11,2,2,1,5,3,2,3,188880,25315,61652.454545,36689.0,62929.509668,2563665.0,830115.0,2.243363e+06,2562619.0,6.976849e+05
4,Train_00004,1,1,1,1,1,1,1,1,36689,36689,36689.000000,36689.0,0.000000,2014022.0,2014022.0,2.014022e+06,2014022.0,0.000000e+00


## 数据合并

In [182]:
train_df = pd.merge(df_base, train_label, on='user', how='left')
train_df = pd.merge(train_df, op_df, on='user', how='left')
train_df = pd.merge(train_df, trans_df, on='user', how='left')

In [183]:
train = train_df

In [184]:
length = len(train)
train_length = int(0.9*length)

x = train.drop(columns=['user','label'],axis=1).values[0:train_length]
y = train['label'].values[0:train_length]

valid_x = train.drop(columns=['user','label'],axis=1).values[train_length:]
valid_y = train['label'].values[train_length:]

In [185]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

# lr = LogisticRegression()
# lr.fit(x,y)

model = LGBMClassifier(n_estimators=500, learning_rate=0.02)
model.fit(x, y,
          eval_set=(valid_x, valid_y),
          early_stopping_rounds=5)

prediction = model.predict_proba(valid_x)[:,1]
auc = roc_auc_score(valid_y,prediction)
auc

[1]	valid_0's binary_logloss: 0.562107
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.561154
[3]	valid_0's binary_logloss: 0.560262
[4]	valid_0's binary_logloss: 0.559423
[5]	valid_0's binary_logloss: 0.558622
[6]	valid_0's binary_logloss: 0.557867
[7]	valid_0's binary_logloss: 0.55718
[8]	valid_0's binary_logloss: 0.556475
[9]	valid_0's binary_logloss: 0.5558
[10]	valid_0's binary_logloss: 0.555142
[11]	valid_0's binary_logloss: 0.554542
[12]	valid_0's binary_logloss: 0.553964
[13]	valid_0's binary_logloss: 0.553421
[14]	valid_0's binary_logloss: 0.552912
[15]	valid_0's binary_logloss: 0.552349
[16]	valid_0's binary_logloss: 0.551791
[17]	valid_0's binary_logloss: 0.551379
[18]	valid_0's binary_logloss: 0.550924
[19]	valid_0's binary_logloss: 0.550531
[20]	valid_0's binary_logloss: 0.550139
[21]	valid_0's binary_logloss: 0.549731
[22]	valid_0's binary_logloss: 0.54937
[23]	valid_0's binary_logloss: 0.548967
[24]	valid_0's binary_logloss: 0.

[217]	valid_0's binary_logloss: 0.529966
[218]	valid_0's binary_logloss: 0.52993
[219]	valid_0's binary_logloss: 0.529894
[220]	valid_0's binary_logloss: 0.529885
[221]	valid_0's binary_logloss: 0.529841
[222]	valid_0's binary_logloss: 0.529791
[223]	valid_0's binary_logloss: 0.529787
[224]	valid_0's binary_logloss: 0.529824
[225]	valid_0's binary_logloss: 0.529791
[226]	valid_0's binary_logloss: 0.529765
[227]	valid_0's binary_logloss: 0.529773
[228]	valid_0's binary_logloss: 0.529737
[229]	valid_0's binary_logloss: 0.529673
[230]	valid_0's binary_logloss: 0.52964
[231]	valid_0's binary_logloss: 0.529652
[232]	valid_0's binary_logloss: 0.529602
[233]	valid_0's binary_logloss: 0.529647
[234]	valid_0's binary_logloss: 0.529619
[235]	valid_0's binary_logloss: 0.52958
[236]	valid_0's binary_logloss: 0.529565
[237]	valid_0's binary_logloss: 0.529553
[238]	valid_0's binary_logloss: 0.529502
[239]	valid_0's binary_logloss: 0.529479
[240]	valid_0's binary_logloss: 0.529484
[241]	valid_0's bin

0.6782946345739088

In [146]:
joblib.dump(model,'model.pkl')

['model.pkl']

In [175]:
pd.DataFrame({
        'column': train.drop(columns=['user','label'],axis=1).columns,
        'importance': model.feature_importances_,
    }).sort_values(by='importance')[-40:]

,column,importance
606,op_mode,47
47,47,47
621,trans_tunnel_out,50
609,net_type,51
607,op_device,52
588,588,52
605,op_type,53
623,trans_ip,53
619,trans_platform,58
22,22,66


baseline:
0.6769352059269422